## 5장-4절. 앙상블의 앙상블 기법

In [92]:
import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
import numpy as np

## 데이터 불러오기

In [93]:
data = pd.read_csv("./data/kc_house_data.csv")
data.head()

,id,date,price,bedrooms,bathrooms,floors,waterfront,condition,grade,yr_built,yr_renovated,zipcode,lat,long
0,7129300520,20141013T000000,221900.0,3,1.00,1.0,0,3,7,1955,0,98178,47.5112,-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2.0,0,3,7,1951,1991,98125,47.7210,-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,1.0,0,3,6,1933,0,98028,47.7379,-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1.0,0,5,7,1965,0,98136,47.5208,-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1.0,0,3,8,1987,0,98074,47.6168,-122.045


In [94]:
data = data.drop(['id','date','zipcode','lat','long'],axis=1)

In [95]:
from sklearn.model_selection import train_test_split
feature_columns = list(data.columns.difference(['price']))

X = data[feature_columns]
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((15129, 8), (6484, 8), (15129,), (6484,))

In [96]:
from sklearn.preprocessing import StandardScaler

ss_scaler = StandardScaler()

X_train_scaled = ss_scaler.fit_transform(X_train)
X_test_scaled = ss_scaler.transform(X_test)

---
---

## lightgbm을 기본모델로 설정

In [97]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

In [99]:
X_train_scaled = pd.DataFrame(X_train_scaled, index=y_train.index)
X_train_scaled.head()

,0,1,2,3,4,5,6,7
13392,-0.469197,-0.412084,-0.626564,-0.915964,-0.554609,-0.084793,-0.130527,-0.210163
9229,0.501558,1.788558,0.918804,0.937650,0.293792,-0.084793,0.175171,-0.210163
14178,-0.469197,0.688237,-0.626564,-0.915964,0.293792,-0.084793,0.209138,-0.210163
10007,0.177973,0.688237,-0.626564,-0.915964,1.142193,-0.084793,-0.198460,-0.210163
3376,-0.469197,-0.412084,0.918804,-0.915964,-0.554609,-0.084793,0.446904,-0.210163


In [100]:
X_test_scaled = pd.DataFrame(X_test_scaled, index=y_test.index)
X_test_scaled.head()

,0,1,2,3,4,5,6,7
1015,-1.439953,-0.412084,-0.626564,-0.915964,-0.554609,-0.084793,0.311038,-0.210163
14367,-1.439953,-2.612726,-2.171933,-0.915964,-1.403011,-0.084793,-2.202485,-0.210163
19025,-0.145612,-1.512405,-0.626564,0.010843,-0.554609,-0.084793,-1.659020,4.778336
3000,0.825143,0.688237,-0.626564,-0.915964,0.293792,-0.084793,0.311038,-0.210163
2377,-0.145612,1.788558,-0.626564,-0.915964,-0.554609,-0.084793,-0.334326,-0.210163


In [101]:
bagging_predict_result = []
for _ in range(10):
    data_index = [data_index for data_index in range(X_train.shape[0])]
    random_data_index = np.random.choice(data_index, X_train.shape[0])
    print(len(set(random_data_index)))
    bg_X_train = X_train_scaled.iloc[random_data_index,]
    bg_y_train = y_train.iloc[random_data_index,]
    lgbm = LGBMRegressor(n_estimators=500,learning_rate=0.01,max_depth=10)
    evals = [(X_test, y_test)]
    lgbm.fit(bg_X_train, bg_y_train, verbose=False)
    lgbm_pred = lgbm.predict(X_test_scaled)
    bagging_predict_result.append(lgbm_pred)
    print(np.sqrt(mean_squared_error(y_test,lgbm_pred)))

9605
209502.35508623754
9478
204271.8841003677
9572
202583.7048758024
9563
203231.36273999474
9498
207739.2487651943
9542
207859.86562394793
9589
202564.16397220947
9646
204001.43135384095
9559
203229.7497710792
9587
207159.26981345282


In [102]:
### 주의 values는 빠져야 한다!!!
# Bagging을 바탕으로 예측한 결과값에 대한 평균을 계산
bagging_predict = [] # 빈 리스트 생성
for lst2_index in range(X_test.shape[0]): # 테스트 데이터 개수만큼의 반복
    temp_predict = [] # 임시 빈 리스트 생성 (반복문 내 결과값 저장)
    for lst_index in range(len(bagging_predict_result)): # Bagging 결과 리스트 반복
        temp_predict.append(bagging_predict_result[lst_index][lst2_index]) # 각 Bagging 결과 예측한 값 중 같은 인덱스를 리스트에 저장
    bagging_predict.append(np.mean(temp_predict))

In [103]:
print(np.sqrt(mean_squared_error(y_test,bagging_predict)))

201393.4939051154
